In [2]:
pip install requests beautifulsoup4 selenium lxml pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.4.0
    Uninstalling urllib3-2.4.0:
      Successfully uninstalled urllib3-2.4.0


In [3]:
import os
import time
import requests
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup
import re

# --------------------------------------
# Clean and extract meaningful text
# --------------------------------------
def clean_text(html_content):
    soup = BeautifulSoup(html_content, 'lxml')
    for script in soup(["script", "style", "noscript"]):
        script.extract()
    text = soup.get_text(separator=' ', strip=True)
    text = re.sub(r'\s+', ' ', text)
    return text

# --------------------------------------
# Save content to .txt file
# --------------------------------------
def save_text(url, text, folder="extracted_texts"):
    os.makedirs(folder, exist_ok=True)
    filename = urlparse(url).path.replace("/", "_")
    if not filename or filename == "_":
        filename = "home"
    with open(os.path.join(folder, f"{filename}.txt"), "w", encoding="utf-8") as f:
        f.write(f"URL: {url}\n\n{text}")

# --------------------------------------
# Crawl function with Requests
# --------------------------------------
def crawl_static(url, visited, depth=0, max_depth=2):
    if url in visited or depth > max_depth:
        return
    visited.add(url)

    try:
        print(f"[Requests] Fetching: {url}")
        response = requests.get(url, timeout=10)
        if response.status_code != 200:
            print(f"Failed to fetch {url}: Status code {response.status_code}")
            return
        text = clean_text(response.text)
        save_text(url, text)
        soup = BeautifulSoup(response.text, 'lxml')

    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch {url} with error: {e}")
        return

    # Recursively crawl other internal links
    base = "{0.scheme}://{0.netloc}".format(urlparse(url))
    for link in soup.find_all('a', href=True):
        href = link['href']
        abs_url = urljoin(base, href)
        if urlparse(abs_url).netloc == urlparse(url).netloc and abs_url.startswith("http"):
            crawl_static(abs_url, visited, depth + 1, max_depth)

# --------------------------------------
# MAIN
# --------------------------------------
if __name__ == "__main__":
    start_url = "https://www.mosdac.gov.in"
    visited = set()
    crawl_static(start_url, visited, max_depth=2)  # You can increase depth for more pages
    print("\n✅ Extraction completed. All texts saved in 'extracted_texts/' folder.")

[Requests] Fetching: https://www.mosdac.gov.in
[Requests] Fetching: https://www.mosdac.gov.in#main-content
[Requests] Fetching: https://www.mosdac.gov.in/internal/registration
[Requests] Fetching: https://www.mosdac.gov.in/internal/uops
[Requests] Fetching: https://www.mosdac.gov.in/internal/logout
[Requests] Fetching: https://www.mosdac.gov.in/
[Requests] Fetching: https://www.mosdac.gov.in/insat-3dr
[Requests] Fetching: https://www.mosdac.gov.in/insat-3d
[Requests] Fetching: https://www.mosdac.gov.in/kalpana-1
[Requests] Fetching: https://www.mosdac.gov.in/insat-3a
[Requests] Fetching: https://www.mosdac.gov.in/megha-tropiques
[Requests] Fetching: https://www.mosdac.gov.in/saral-altika
[Requests] Fetching: https://www.mosdac.gov.in/oceansat-2
[Requests] Fetching: https://www.mosdac.gov.in/oceansat-3
[Requests] Fetching: https://www.mosdac.gov.in/insat-3ds
[Requests] Fetching: https://www.mosdac.gov.in/scatsat-1
[Requests] Fetching: https://www.mosdac.gov.in/internal/catalog-satellite

/tmp/ipython-input-3-1925126326.py:12: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(html_content, 'lxml')
/tmp/ipython-input-3-1925126326.py:46: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing mi

[Requests] Fetching: https://www.mosdac.gov.in/docs/STQC.pdf
[Requests] Fetching: https://www.mosdac.gov.in/mosdac-feedback
[Requests] Fetching: https://www.mosdac.gov.in/about-us
[Requests] Fetching: https://www.mosdac.gov.in/contact-us
[Requests] Fetching: https://www.mosdac.gov.in/copyright-policy
[Requests] Fetching: https://www.mosdac.gov.in/data-access-policy
[Requests] Fetching: https://www.mosdac.gov.in/hyperlink-policy
[Requests] Fetching: https://www.mosdac.gov.in/privacy-policy
[Requests] Fetching: https://www.mosdac.gov.in/website-policies
[Requests] Fetching: https://www.mosdac.gov.in/terms-conditions
[Requests] Fetching: https://www.mosdac.gov.in/faq-page

✅ Extraction completed. All texts saved in 'extracted_texts/' folder.


In [6]:
import os
import spacy
import pandas as pd
from bs4 import BeautifulSoup

# Load spaCy NLP model
nlp = spacy.load("en_core_web_sm")

# Input folders
text_folder = "extracted_texts"

# Store all extracted triples
triples = []

# --------------------- Extract Clean Text from HTML --------------------- #
def extract_text_from_html(filepath):
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f.read(), "lxml")
            for tag in soup(["script", "style", "noscript"]):
                tag.extract()
            return soup.get_text(separator=' ', strip=True)
    except Exception as e:
        print(f"❌ Error reading {filepath}: {e}")
        return ""

# --------------------- Extract Triples Using spaCy --------------------- #
def extract_triples_from_text(text):
    doc = nlp(text)
    for sent in doc.sents:
        subject = predicate = obj = ""
        for token in sent:
            if "subj" in token.dep_:
                subject = token.text
            elif token.dep_ == "ROOT":
                predicate = token.lemma_
            elif "obj" in token.dep_:
                obj = token.text
        if subject and predicate and obj:
            yield (subject, predicate, obj)

# --------------------- Process All Text Files --------------------- #
def process_text_files(folder_path):
    print(f"🔍 Processing text files in {folder_path}")
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            try:
                with open(os.path.join(folder_path, filename), "r", encoding="utf-8") as f:
                    text = f.read()
                    for triple in extract_triples_from_text(text[:3000]):  # Limit per file
                        yield triple
            except Exception as e:
                print(f"⚠️ Skipped {filename}: {e}")

# --------------------- Process All HTML Files --------------------- #
def process_html_files(folder_path):
    print(f"🔍 Processing HTML files in {folder_path}")
    for filename in os.listdir(folder_path):
        if filename.endswith(".html"):
            filepath = os.path.join(folder_path, filename)
            text = extract_text_from_html(filepath)
            for triple in extract_triples_from_text(text[:3000]):  # Limit per file
                yield triple

# --------------------- Main Execution --------------------- #
if __name__ == "__main__":
    seen_triples = set()

    for triple in process_text_files(text_folder):
        if triple not in seen_triples:
            triples.append(triple)
            seen_triples.add(triple)

    # Save to CSV
    df = pd.DataFrame(triples, columns=["Subject", "Predicate", "Object"])
    df.to_csv("knowledge_graph_triples.csv", index=False)
    print(f"\n✅ Saved {len(triples)} unique triples to 'knowledge_graph_triples.csv'")

🔍 Processing text files in extracted_texts

✅ Saved 352 unique triples to 'knowledge_graph_triples.csv'


In [7]:
import pandas as pd

# Load your triples
df = pd.read_csv("knowledge_graph_triples.csv")

# Create searchable sentences from triples
df["sentence"] = df["Subject"] + " " + df["Predicate"] + " " + df["Object"]
df.to_csv("kg_sentences.csv", index=False)
print("✅ Created searchable sentence dataset: 'kg_sentences.csv'")

✅ Created searchable sentence dataset: 'kg_sentences.csv'


In [8]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 71.4 MB/s eta 0:00:00


In [9]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Load pre-trained semantic model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Load sentences
df = pd.read_csv("kg_sentences.csv")
sentences = df["sentence"].tolist()

# Encode all sentences
embeddings = model.encode(sentences, convert_to_numpy=True)

# Create FAISS index for fast similarity search
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

print("✅ Semantic index created with FAISS.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Semantic index created with FAISS.


In [10]:
def get_top_k_matches(query, k=3):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, k)
    results = []
    for i in indices[0]:
        results.append(df.iloc[i]["sentence"])
    return results

# Example Query
query = "Which satellite gives rainfall data?"
results = get_top_k_matches(query)

print("🔍 Top Matches:")
for r in results:
    print("-", r)

🔍 Top Matches:
- satellite commission data
- information be weather
- Bayesian base rainfall


In [11]:
import pandas as pd

# Load the CSV containing Subject, Predicate, Object, and Sentence
df = pd.read_csv("kg_sentences.csv")

# If 'sentence' column doesn't exist, create it from Subject–Predicate–Object
if "sentence" not in df.columns:
    df["sentence"] = df["Subject"] + " " + df["Predicate"] + " " + df["Object"]

In [12]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load a compact sentence transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Encode all sentences into vectors
sentences = df["sentence"].tolist()
embeddings = model.encode(sentences, convert_to_numpy=True)

# Create a FAISS index (flat L2 distance)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

print(f"✅ Knowledge base built with {len(sentences)} entries.")

✅ Knowledge base built with 352 entries.


In [13]:
def search_knowledge_graph(query, k=3):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, k)
    results = []
    for i in indices[0]:
        results.append(df.iloc[i]["sentence"])
    return results

In [14]:
query = "How can I access satellite rainfall data?"
answers = search_knowledge_graph(query, k=3)

print(f"🔍 Query: {query}\n💡 Top Matches:")
for ans in answers:
    print("→", ans)

🔍 Query: How can I access satellite rainfall data?
💡 Top Matches:
→ satellite commission data
→ information be weather
→ Bayesian base rainfall
